In [61]:
import argparse
import copy
import random

In [62]:
parser = argparse.ArgumentParser(description='Auto-generate COOL code.')
parser.add_argument('--random_seed', type=int, dest='random_seed', default=0,
                   help='Random seed integer for consistency.')
parser.add_argument('--gen_attr_prob', type=float, dest='gen_attr_prob', default=0.8,
                   help='Probability another attribute is created.')
parser.add_argument('--gen_func_prob', type=float, dest='gen_func_prob', default=0.8,
                   help='Probability another function is created.')
parser.add_argument('--gen_func_args_prob', type=float, dest='gen_func_args_prob', default=0.8,
                   help='Probability another argument in a function is created.')
parser.add_argument('--gen_class_prob', type=float, dest='gen_class_prob', default=0.8,
                   help='Probability another class is created')

args = parser.parse_args(['--random_seed', '789'])
if args.random_seed != 0:
    random.seed(args.random_seed)

In [63]:
# convenience functions
class Id(object):
    def __init__(self, prefix):
        self.prefix = prefix
        self.count = -1

    def get_id(self):
        self.count += 1
        return self.prefix + str(self.count)
    
    def reset(self):
        self.count = -1

In [77]:
class Class(object):
    def __init__(self, name, parent_class):
        assert (parent_class != None), "Cannot replace root class."
        self.name = name
        self.parent = parent_class
        
        # default initializations
        self.attributes = copy.deepcopy(self.parent.attributes)
        self.attr_types = copy.deepcopy(self.parent.attr_types)
        self.basic = False
        self.functions = None
        self.func_type_dict = dict() if self.parent == None else copy.deepcopy(self.parent.func_type_dict)
        self.inheritable = True

    def gen_attr_signatures(self, classes):
        if self.basic == True: 
            print "error: not allowed to change basic classes"
            return None
        self.attributes = {'self': None}
        attr_id = Id('attr')
        while random.random() < args.gen_attr_prob:
            self.attributes[attr_id.get_id()] = None
    
    def gen_attr_expressions(self, classes, expr_factory):
        for attr in self.attributes:
            if self.attributes[attr] == None:
                self.attributes[attr] = expr_factory.gen_expression(self)

    def gen_func_signatures(self, classes):
        if self.basic == True:
            print "error: not allowed to change basic classes"
            return None
        self.func_dict = list()
        self.func_type_dict = list()
        
        func_id = Id('func')
        while random.random() < args.gen_attr_prob:
            func_name = func_id.get_id()
            func_type = random.choice(classes).name
            func_args = list()
            arg_id = Id('arg')
            while random.random() < args.gen_func_args_prob:
                arg_name = arg_id.get_id()
                arg_type = random.choice(classes).name
                func_args.append((arg_name, arg_type))
            self.functions.append(Function(func_name, func_type, func_args))
    
    def gen_func_expressions(self, classes, expr_factory):
        if self.attributes == None:
            print "error: must initialize attributes before function expressions"
            return None
        for func in self.functions:
            if func.expr != None: # don't overwrite a defined class-specific method
                continue 
            self.func.expr = expr_factory.gen_expression(self) 
        
    def pprint(self):
        next_indent = '   '
        print 'class', self.name, 
        if self.parent != None:
            print 'inherits', self.parent.name, 
        print '{'
        for attr in self.attributes:
            if attr != 'self':
                print next_indent, attr, ':', self.attributes[attr].expr_type, ';'
        for func_name in self.functions:
            self.functions[func_name].pprint(next_indent)
        print '};'
        
    @staticmethod
    def get_attr_types(c):
        try:
            return c.attr_types
        except AttributeError:
            attr_types = dict() if self.parent == None else copy.deepcopy(self.parent.attr_types)
            for attribute in c.attributes:
                attr_type = c.attributes[attribute].expr_type
                attr_types[attr_type] = attr_types.get(attr_type, list()).extend([attribute])
            return attr_types

    @staticmethod
    def get_func_types(c):
        try:
            return c.func_types
        except AttributeError:
            func_types = dict()
            for function in c.functions:
                func_name = function.name
                func_type = function.return_type
                func_types[func_type] = func_types.get(func_type, list()).extend([func_name])
            return func_types
        
class Expression(object):
    def __init__(self, expr=None):
        if expr == None:
            self.expr_type = ''
            self.expr_terms = None
        else:
            self.expr_type = expr[0]
            self.expr_terms = expr[1]

    def pprint(self, indent=''):
        if self.expr_type != '' and self.expr_terms != None:
            print indent, ' '.join(self.expr_terms)
    
class Function(object):
    def __init__(self, name, return_type, args=list(), expr=Expression()):
        self.return_type = return_type
        self.name = name
        self.args = args
        self.expr = expr
        
    def pprint(self, indent=''):
        next_indent = indent + '   '
        print indent, self.name, '(',
        for i, arg in enumerate(self.args):
            if i > 0: 
                print ',',
            print arg[0], ':', arg[1],
        print ')', ':', self.return_type, '{'
        self.expr.pprint(next_indent)
        print indent, '};'


In [82]:
class Object(Class):
    def __init__(self):
        self.name = 'Object'
        self.parent = None
        self.attributes = {'self' : Expression(('SELF_TYPE','void'))}
        self.attr_types = Class.get_attr_types(self)
        self.basic = True
        self.functions = {
            'abort' : Function('abort', 'Object'),
            'type_name' : Function('type_name', 'String'), 
            'copy' : Function('copy', 'SELF_TYPE')
        }
        self.func_types = Class.get_func_types(self)
        self.inheritable = True

class IO(Class):
    def __init__(self, parent_class=Object()):
        self.name = 'IO'
        self.parent = parent_class
        self.attributes = {'self': Expression(('SELF_TYPE','void'))}
        self.attr_types = Class.get_attr_types(self)
        self.basic = True
        self.functions = {
            'out_string' : Function('out_string', 'SELF_TYPE', [('x', 'String')]),
            'out_int' : Function('out_int', 'SELF_TYPE', [('x', 'Int')]),
            'in_string' : Function('in_string', 'String'),
            'in_int' : Function('in_int', 'Int')
        } 
        self.func_types = Class.get_func_types(self)
        self.inheritable = True

class Int(Class):
    def __init__(self, parent_class=Object()):
        self.name = 'Int'
        self.parent = parent_class
        self.attributes = {'self': Expression(('SELF_TYPE','0'))} # default value = 0
        self.attr_types = Class.get_attr_types(self)
        self.basic = True
        self.functions = list()
        self.func_types = Class.get_func_types(self)
        self.inheritable = False

class String(Class):
    def __init__(self, parent_class=Object()):
        self.name = 'String'
        self.parent = parent_class
        self.attributes = {'self': Expression(('SELF_TYPE',''))} # default value = empty string
        self.attr_types = Class.get_attr_types(self)
        self.basic = True
        self.functions = {
            Function('length', 'Int'),
            Function('concat', 'String', [('x', 'String')]),
            Function('substr', 'String', [('i', 'Int'), ('l', 'Int')])}
        self.func_types = Class.get_func_types(self)
        self.inheritable = False

class Bool(Class):
    def __init__(self, parent_class=Object()):
        self.name = 'Bool'
        self.parent = parent_class
        self.attributes = {'self': Expression(('SELF_TYPE','false'))} # default value = false
        self.attr_types = Class.get_attr_types(self)
        self.basic = True
        self.functions = list()
        self.func_types = Class.get_func_types(self)
        self.inheritable = False

AttributeError: 'NoneType' object has no attribute 'attr_types'

In [83]:
class Assign(Expression):
    def gen_expr(self, expr_type, expr_factory)
        self.expr_type = expr_type
        var = expr_factory.gen_variable(expr_type)
        e1 = expr_factory.gen_expression(expr_type)
        self.expr_terms = [var, '<-', e1]

class New(Expression):
    def gen_expr(self, expr_type, expr_factory):
        self.expr_type = expr_type
        self.expr_terms = ['new', expr_type]
    
# class Dispatch(Expression):
# class Static_dispatch(Expression):

class If(Expression):
    def gen_expr(self, expr_type, expr_factory):
        self.expr_type = expr_type
        e1 = expr_factory.gen_expression('Bool')
        e2 = expr_factory.gen_expression(expr_type)
        e3 = expr_factory.gen_expression(expr_type)
        self.expr_terms = ['if', e1, 'else', e2, 'fi']

class Sequence(Expression):
    def gen_expr(self, expr_type, expr_factory):
        self.expr_type = expr_type
        self.expr_terms = ['{']
        while random.random() < 0.5:
            eI = expr_factory.gen_expression() # random expression
            self.expr_terms.extend([eI, ';'])
        eN = expr_factory.gen_expression(expr_type)
        self.expr_terms.extend([eN, ';', '}'])
        
class Let_init(Expression):
    def gen_expr(self, expr_type, expr_factory):
        self.expr_type = expr_type
        # TODO: unfinished

# class Let_no_init(Expression):
# class Case(Expression):

class Loop(Expression):
    def gen_expr(self, expr_type, expr_factory):
        self.expr_type = expr_type
        e1 = expr_factory.gen_expression('Bool')
        e2 = expr_factory.gen_expression(expr_type)
        self.expr_terms = ['while', e1, 'loop', e2, 'pool']

class Isvoid(Expression):
    def __init__(self):
        self.expr_type = 'Bool'
        self.expr_terms = None
    
    def gen_expr(self, expr_type, expr_factory):
        assert (expr_type == 'Bool'), "creating non-boolean isvoid expression"
        e1 = expr_factory.gen_expression() # random expression
        self.expr_terms = ['isvoid', e1]
    
class Not(Expression):
    def __init__(self):
        self.expr_type = 'Bool'
        self.expr_terms = None
    
    def gen_expr(self, expr_type, expr_factory):
        assert (expr_type == 'Bool'), "creating non-boolean not expression"
        e1 = expr_factory.gen_expression('Bool')
        self.expr_terms = ['not', e1]

class Compare(Expression):
    def __init__(self):
        self.expr_type = 'Bool'
        self.expr_terms = None

    def gen_expr(self, expr_type, expr_factory):
        assert (expr_type == 'Bool'), "creating non-boolean compare expression"
        e1 = expr_factory.gen_expression('Int')
        e2 = expr_factory.gen_expression('Int')
        self.expr_terms = [e1, '<', e2]

class Neg(Expression):
    def __init__(self):
        self.expr_type = 'Bool'
        self.expr_terms = None

    def gen_expr(self, expr_type, expr_factory):
        assert (expr_type == 'Bool'), "creating non-boolean neg expression"
        e1 = expr_factory.gen_expression('Bool')
        self.expr_terms = ['~', e1]
    
# class Arith(Expression):
class Var(Expression):
    def gen_expr(self, expr_type, expr_factory):
        self.expr_type = expr_type
        e1 = expr_factory.gen_variable(expr_type)
        self.expr_terms = [e1]

# class Equal(Expression):

SyntaxError: invalid syntax (<ipython-input-83-bd150e4e4e3d>, line 2)

In [84]:
class Expression_factory(object):
    def __init__(self, classes):
        self.classes = classes
        self.expressions = {
            'Assign' : Assign(), 
            'New' : New(), 
            'Dispatch' : Dispatch(), 
            'Static_dispatch' : Static_dispatch(), 
            'If' : If(), 
            'Sequence' : Sequence(), 
            'Let_init' : Let_init(), 
            'Let_no_init' : Let_no_init(), 
            'Case' : Case(), 
            'Loop' : Loop(), 
            'Isvoid' : Isvoid(), 
            'Not' : Not(), 
            'Compare' : Compare(), 
            'Neg' : Neg(), 
            'Arith' : Arith(), 
            'Var' : Var(), 
            'Equal' : Equal()}
        
    def init_expressions(self):
        expressions 
        
    def gen_expression(self, expr_type):
        return Expression()

class Class_factory(object):
    def __init__(self):
        self.classes = list()
        self.expr_factory = None
        
        # initialize the classes
        self.init_basic_classes()
        self.gen_classes()

    def init_basic_classes(self):
        root_class = Object()
        basic_classes = [root_class, IO(root_class), Int(root_class), String(root_class), Bool(root_class)]
        self.classes.extend(basic_classes)
    
    def gen_classes(self):
        # 'flip a coin' and create a class when heads/true
        class_id = Id('class')
        while random.random() < args.gen_class_prob:
            class_name = class_id.get_id()
            parent_class = random.choice(self.classes)
            while not parent_class.inheritable:
                parent_class = random.choice(self.classes)
            self.classes.append(Class(class_name, parent_class))

        # initialize expression factory
        self.expr_factory = Expression_factory(self.classes)
            
        # initialize in order: prototypes => definitions
        # because it allows for attribute initializations and func definitions 
        # to use other attributes, functions (maybe recursion), which makes things 
        # things trickier
        for c in self.classes:
            if c.basic != True:
                c.gen_func_signatures(self.classes)
                c.gen_attr_signatures(self.classes)
        for c in self.classes:
            if c.basic != True:
                c.gen_func_expressions(self.classes, self.expr_factory)
                c.gen_attr_expressions(self.classes, self.expr_factory)

In [85]:
class_factory = Class_factory()
class_factory.classes

# for c in class_factory.classes:
#     c.pprint()
#     print " "

AttributeError: 'NoneType' object has no attribute 'attr_types'

In [58]:
class dummy:
    i = 0

In [59]:
d = dummy()
d.i

0

In [60]:
d.j

AttributeError: dummy instance has no attribute 'j'